# Analysing water levels of the river Isar

In light of the recent flooding catastrophe with more then 170 dead, my interest sparked in monitoring and maybe predicting the waterlevels of the river in my hometown Munich, the river Isar.

In this project you will get an idea how i refine my workflow and the identification of interesting data. I will also improve my skills in webscraping with beautiful soup and time series analysis.

The project is structured in three main parts:

1. Problem formulation and subject understanding
2. Datasource identification and webscraping
3. Data wrangling and time series analysis

## Project goal and motivation

I started this project to get improve and expand two skillsets in my data science toolbox:

Tools and workflow technologies
- Visual Studio Code
- Github
- Python scripting

Data science subject
- Time Series analysis
- Time Series forecasting

Also i see it as a portfolio project, highlighting my current skills in the above mentioned subjects, story telling and general Python programming. I will probably  
use a lot ouf resources from practitioners, learners, professionals and amateures. For this reason i want to share my project and the isights i've gained with you and  
i'm more then curios about your comments, suggestions and enhancements.

Acknowledgements

## 0. Loading packages

In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from datetime import datetime, timedelta
from datetime import date

now = datetime.today()

import os
from os import walk
dirname = os.getcwd()


In [148]:
#Script handling parameters
scraping = False
concatenating = True
deleting_scraped_files = False
save_to = 'CSV'

## 1. Problem formulation and subject understanding

The isar is a 292,3 kilometer long river that originates in the very north of Austria. The river crosses the capital of bavaria Munich until it merges with the Donau river south of Deggendorf. [Source: wikipedia.com/isar]



# 1. Data collecting phase

## 1.1. Data Source identification

The bavarian ministry for envoironment runs the so called "Hochwassernachrichtendienst" HND where the water levels of several rivers and lakes are provided. Additionally there are information on precipication and others.

We focus on scraping the water levels first. The site is structured as follows

    Basic Link:           https://www.hnd.bayern.de/pegel/meldestufen/isar/tabellen
    Additional Parameter: ?days=0&hours=1

We can address the levels per day for the last 30 days while 0 is today and 29 the oldest. The data is provided on an hourly interval from 0 to 23. For now, we don't need the most recent data, but this might change when upgrading to a more sophisticated scraping approach. At the moment we focus on getting a basic scrapping to work.

In [149]:
#Web scraping properties
#link = "https://www.hnd.bayern.de/pegel/meldestufen/isar/tabellen?days=0&hours=1"
basic_link = "https://www.hnd.bayern.de/pegel/tabellen"

## 1.2. Web scraping

Currently we scrap the full month of data and discarge the duplicates when we merge the DataFrames. This leads to a unneccessary long scraping time.  
In order to reduce the scapring time we want to determine which days need to be scraped by comparing the difference between the recent data and the   
current date.

In [150]:
#list history file and read as df
for file in os.listdir():
    if 'bay_river_pegel' in file:
        hist_pegel = pd.read_csv(file)
        last_scraped_datetime = pd.to_datetime(hist_pegel['Datum Zeit'][0], format='%Y-%m-%d %H:%M:%S')
        
        #determine the range of days to scrape
        days_scrape  = [i for i in range(0,(now.day - last_scraped_datetime.day + 1))]
    else:
        #full scrape if no history available
        hist_pegel = None
        days_scrape  = [i for i in range(0,31)]


#full hours to scrape
hours_scrape = [i for i in range(0,24)]


In [151]:
#File handling properties
von_string = str(date.today() - timedelta(days_scrape[-1]))
bis_string = str(date.today() - timedelta(days_scrape[0]))
path = dirname + "/ScrapingData/"

save_string = 'bay_river_pegel' + '_' + von_string +  '_bis_' + bis_string

As you can see, pandas 'read_html' class is a very powerful tool to get html tabels quickly into a dataframe. We now automate this approach to scrape the full history of the water levels.

In [152]:
#Web scraping
if scraping:

    df_water_levels_scrape = pd.DataFrame()
    
    #add handling for most recent data
    for day in days_scrape:
        #when today, only scrape to current hour
        if day == 0:

            for hour in hours_scrape:
                if hour <= (now.hour - 1):
                    scrape_link = basic_link + '?days=' + str(day) + '&hours=' + str(hour)
                    water_level = pd.read_html(scrape_link)
                    df_water_levels_scrape = df_water_levels_scrape.append(water_level[0])
                    
        #when past days, scrape full hours
        else:
            for hour in hours_scrape:
                scrape_link = basic_link + '?days=' + str(day) + '&hours=' + str(hour)
                water_level = pd.read_html(scrape_link)
                df_water_levels_scrape = df_water_levels_scrape.append(water_level[0])

    #df_water_levels_scrape['Datum Zeit'] = pd.to_datetime(df_water_levels_scrape['Datum Zeit'], format='%d.%m.%Y, %H:%M')
    df_water_levels_scrape.to_csv((path + save_string + '.csv'), index=False)

In [153]:
#concat files in the scraping folder into a single df

if concatenating == True:
    #list files in the scraping folder and concat to single df
    scraped_files       = os.listdir(path)
    df_from_each_file   = (pd.read_csv(path + f) for f in scraped_files)
    concatenated_df     = pd.concat(df_from_each_file, axis=0, ignore_index=True)

    #search for non available data and mark as nan. We do this step only on newly scraped data
    missing_string = 'Derzeit leider keine aktuellen Daten vorhanden.'
    concatenated_df = concatenated_df[concatenated_df != missing_string]
    
    #Subset because there are NaN in column "Vorhersage", which would drop everything
    concatenated_df = concatenated_df.dropna(subset=['Wasser­stand [cm]'])

    #sort and drop duplicates
    concatenated_df     = concatenated_df.sort_values(by=['Datum Zeit'], ascending = True)
    concatenated_df     = concatenated_df.drop_duplicates(['Messstelle','Datum Zeit'])

    #concat hist and recent scraping
    concatenated_df_2     = pd.concat([hist_pegel, concatenated_df], axis=0, ignore_index=True)
    concatenated_df_2     = concatenated_df_2.sort_values(by=['Datum Zeit'], ascending = True)

    #Clean up by dropping duplicates in observation point and datetime
    concatenated_df_2   = concatenated_df_2.drop_duplicates(['Messstelle','Datum Zeit'])
    concatenated_df_2['Datum Zeit'] = pd.to_datetime(concatenated_df_2['Datum Zeit'], format='%d.%m.%Y, %H:%M')
    concatenated_df_2     = concatenated_df_2.sort_values(by=['Datum Zeit'], ascending = False)

Since our scraping works as intended, we take the time to investigate the outcome. In theory, there should be 24 obersvations per day and oberservation point. We check this by saving an oberservation point to a dedicated DataFrame and group by date.

In [168]:
waterlevel_sylvenstein = concatenated_df_2[concatenated_df_2['Messstelle'] == 'Sylvenstein']
waterlevel_sylvenstein.groupby(by=waterlevel_sylvenstein['Datum Zeit'].dt.date).count()

,Messstelle,Gewässer,Datum Zeit,Wasser­stand [cm],Änderung seit 2 Std. [cm],Abfluss [m³/s],Melde­stufe,Jähr­lichkeit,Vorher­sage
Datum Zeit,,,,,,,,,
2021-08-23,24,24,24,24,24,24,24,24,0
2021-08-24,24,24,24,24,24,24,24,24,0
2021-08-25,24,24,24,24,24,24,24,24,0
2021-08-26,24,24,24,24,24,24,24,24,0
2021-08-27,24,24,24,24,24,24,24,24,0
2021-08-28,24,24,24,24,24,24,24,24,0
2021-08-29,24,24,24,24,24,24,24,24,0
2021-08-30,24,24,24,24,24,24,24,24,0
2021-08-31,24,24,24,24,24,24,24,24,0


The scraped data is indeed what we have expected, exept for the last day, which is the current day and scraping is still a work in progress.

Lastly we take a look at a complete day and see if we spot any mistakes whatsoever.

In [165]:
waterlevel_sylvenstein

,Messstelle,Gewässer,Datum Zeit,Wasser­stand [cm],Änderung seit 2 Std. [cm],Abfluss [m³/s],Melde­stufe,Jähr­lichkeit,Vorher­sage
133568,Sylvenstein,Isar,2021-09-22 19:00:00,242,0,116,0,---,NaN
133318,Sylvenstein,Isar,2021-09-22 18:00:00,242,0,116,0,---,NaN
133078,Sylvenstein,Isar,2021-09-22 17:00:00,242,0,116,0,---,NaN
132826,Sylvenstein,Isar,2021-09-22 16:00:00,242,0,116,0,---,NaN
132574,Sylvenstein,Isar,2021-09-22 15:00:00,242,0,116,0,---,NaN
...,...,...,...,...,...,...,...,...,...
134606,Sylvenstein,Isar,2021-08-23 04:00:00,256,0,192,0,---,NaN
134584,Sylvenstein,Isar,2021-08-23 03:00:00,256,0,192,0,---,NaN
134328,Sylvenstein,Isar,2021-08-23 02:00:00,256,+1,192,0,---,NaN
134074,Sylvenstein,Isar,2021-08-23 01:00:00,256,0,192,0,---,NaN


We are happy with the results and save the dataframe to the disk.

### Persistently save data

Obviously, we want to save our current efforts persistently to save the historical data we have put so much thought and effort in. We use .csv for now and also delete the last complete file to keep things tidy.

In [169]:
#delete current history file
if 'bay_river_pegel' in file:
    os.remove(file)

#determine filename from content taking into account that values could be missing
end_date            = concatenated_df_2['Datum Zeit'].iloc[0].strftime('%Y-%m-%d')
start_date          = concatenated_df_2['Datum Zeit'].iloc[-1].strftime('%Y-%m-%d')
save_string2        = 'bay_river_pegel' + '_' + start_date +  '_bis_' + end_date

#saving concatenated df as .csv to disk
concatenated_df_2.to_csv((save_string2 + '.csv'), index=False)

However with saving to .csv we loose the data type information from our DataFrame. For example, the datetime column would be loaded as object and not as datetime object.
With the use of pythons pickle format (.pkl) these information is also saved to the file, allowing us to efficiently load the date in later steps. We take this opportunity
to further refine our datatypes by looking at the current formats and see if we can optimize our data further.

In [170]:
concatenated_df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188623 entries, 133554 to 133594
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Messstelle                 188623 non-null  object        
 1   Gewässer                   188623 non-null  object        
 2   Datum Zeit                 188623 non-null  datetime64[ns]
 3   Wasser­stand [cm]          188623 non-null  object        
 4   Änderung seit 2 Std. [cm]  188623 non-null  object        
 5   Abfluss [m³/s]             188623 non-null  object        
 6   Melde­stufe                188623 non-null  object        
 7   Jähr­lichkeit              188623 non-null  object        
 8   Vorher­sage                0 non-null       object        
dtypes: datetime64[ns](1), object(8)
memory usage: 14.4+ MB


In [158]:
os.stat(save_string2 + '.csv')

os.stat_result(st_mode=33188, st_ino=37267953, st_dev=16777220, st_nlink=1, st_uid=501, st_gid=20, st_size=10822642, st_atime=1632335785, st_mtime=1632335785, st_ctime=1632335785)

In [159]:
#deleting scraped data
if deleting_scraped_files == True:
    for f in scraped_files:
        os.remove(path + f)


Since we are now able to automatically scrape and save the waterlevels, lets take a look at the date we get.

# 2. Data understanding phase

Before we can work with the data we need to understand the collected data first. I have separated this task into two main parts;

- *The quantitaive data understanding*  
Here we want to now how the data looks in the first place. Which variabels are in the data, how many observations, NaN values, data types, 
statistical information in numerical and unique values in categorical variables.    
  
  
- *The qualitative data understanding*  
Goal is to really understand the data from an domain point of view. How are different rivers, measuring points etc. are connected, what do cetrain variables mean.
These questions are best answered by consulting the website (FAQ, reading material, documentation etc.) or interviewing experts in the field. These insights will
help us further improve further analysis and forecasting.

We start bei loading the data in a DataFrame.

In [160]:
#Load the complete csv to dataframe
for file in os.listdir():
    if 'isar_pegel_' in file:
        waterlevel_hist = pd.read_csv(file)
    
    else:
        waterlevel_hist = None


In [161]:
waterlevel_hist.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
waterlevel_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27125 entries, 0 to 27124
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Messstelle                 27125 non-null  object 
 1   Gewässer                   27125 non-null  object 
 2   Datum Zeit                 27125 non-null  object 
 3   Wasser­stand [cm]          27125 non-null  int64  
 4   Änderung seit 2 Std. [cm]  27125 non-null  int64  
 5   Abfluss [m³/s]             27125 non-null  object 
 6   Melde­stufe                27125 non-null  int64  
 7   Jähr­lichkeit              27125 non-null  object 
 8   Vorher­sage                0 non-null      float64
dtypes: float64(1), int64(3), object(5)
memory usage: 1.9+ MB


In [ ]:
waterlevel_hist['Datum Zeit'] = pd.to_datetime(waterlevel_hist['Datum Zeit'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
waterlevel_messtellen = waterlevel_hist['Messstelle'].unique()
waterlevel_gewässer = waterlevel_hist['Gewässer'].unique()
waterlevel_meldestufe = waterlevel_hist['Melde­stufe'].unique()


In [ ]:
print(waterlevel_messtellen)
print(waterlevel_gewässer)
print(waterlevel_meldestufe)

['Partenkirchen (alt)' 'Garmisch o. d. Partnachmündung' 'Fürstenfeldbruck'
 'Oberammergau' 'Inkofen' 'Ampermoching' 'Stegen' 'Beuerberg' 'Peißenberg'
 'Weilheim' 'Raisting' 'Oberfinning Seepegel' 'Kochel' 'Schlehdorf'
 'Garmisch u. d. Partnachmündung' 'Oberfinning Speicherabgabe' 'Plattling'
 'Sylvenstein' 'Leutstetten' 'Hohenkammer' 'Berg' 'Eschenlohe Brücke'
 'Starnberg' 'Eching' 'Landau' 'Lenggries' 'Puppling' 'München' 'Freising'
 'Landshut Birket' 'Bad Tölz Brücke']
['Partnach' 'Loisach' 'Amper' 'Ammer' 'Rott' 'Windachspeicher' 'Windach'
 'Isar' 'Würm' 'Glonn' 'Sempt' 'Starnberger See' 'Ammersee']
[0 4 1 3 2]


In [ ]:
waterlevel_isar = waterlevel_hist[waterlevel_hist['Gewässer'] == 'Isar']

In [ ]:
#datum_list = waterlevel_sylvenstein['Datum Zeit'].dt.floor('d').value_counts()()



In [ ]:
waterlevel_isar['Messstelle'].unique()

array(['Plattling', 'Sylvenstein', 'Landau', 'Lenggries', 'Puppling',
       'München', 'Freising', 'Landshut Birket', 'Bad Tölz Brücke'],
      dtype=object)